In [1]:
import transformers
import textwrap
from transformers import LlamaForCausalLM, LlamaTokenizer
import os
import sys
from typing import List
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)
import fire
import torch
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from pylab import rcParams
import json

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

/home/ubuntu/Documents/LLM/Llama-lora/chaingpt_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/ubuntu/Documents/LLM/Llama-lora/chaingpt_env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121_nocublaslt.so...


/home/ubuntu/Documents/LLM/Llama-lora/chaingpt_env/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('@1=\\E[1~'), PosixPath('ct=\\E[3g'), PosixPath('ac=\\140\\140aaffggjjkkllmmnnooppqqrrssttuuvvwwxxyyzz{{||}}~~..--++,,hhII00'), PosixPath('AB=\\E[4%dm'), PosixPath('k0=\\E[10~'), PosixPath('as=\\E(0'), PosixPath('dc=\\E[P'), PosixPath('ae=\\E(B'), PosixPath('k4=\\EOS'), PosixPath('us=\\E'), PosixPath('k7=\\E[18~'), PosixPath('cd=\\E[J'), PosixPath('cr=^M'), PosixPath('kN=\\E[6~'), PosixPath('ta=^I'), PosixPath('al=\\E[L'), PosixPath('up=\\EM'), PosixPath('pa#64'), PosixPath('AL=\\E[%dL'), PosixPath('kr=\\EOC'), PosixPath('k9=\\E[20~'), PosixPath('xv'), PosixPath('k6=\\E[17~'), PosixPath('xn'), PosixPath('RI=\\E[%dC'), PosixPath('dl=\\E[M'), PosixPath('k1=\\EOP'), PosixPath('im=\\E[4h'), PosixPath('sr=\\EM'), PosixPath('ti=\\E[?1049h'), PosixPath('DO=\\E[%dB'), Po

In [2]:
BASE_MODEL = "decapoda-research/llama-7b-hf"

In [3]:
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float64,
    device_map="auto", 
    cache_dir = "./Llama_weights"

)

tokenizer = LlamaTokenizer.from_pretrained(
    BASE_MODEL,
    cache_dir = "./Llama_weights"
)

tokenizer.pad_token_id=(0)
tokenizer.padding_side="left"

Overriding torch_dtype=torch.float64 with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:13<00:00,  2.49it/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [4]:
data = load_dataset("json", data_files="all_question_answer_dataset.json")
data["train"]

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/json/default-64a2dfcf18e5361f/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 352.94it/s]


Dataset({
    features: ['Instruction', 'input', 'output'],
    num_rows: 80216
})

In [5]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["Instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""

In [6]:
cutoff_len = 256
def tokenize(prompt, add_eos_token=True):
        # there's probably a way to do this with the tokenizer settings
        # but again, gotta move fast
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

def generate_and_tokenize_prompt(data_point):
    full_prompt=generate_prompt(data_point)
    tokenized_full_prompt=tokenize(full_prompt)
    return tokenized_full_prompt

In [7]:
train_val = data["train"].train_test_split(
    test_size=5000,
    shuffle=True
)

train_data = train_val["train"].shuffle().map(generate_and_tokenize_prompt)
val_data = train_val["test"].shuffle().map(generate_and_tokenize_prompt)

LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 3
OUTPUT_DIR = "experiments"

In [8]:
model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


In [9]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    load_best_model_at_end=True
)

data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        )

In [10]:
trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=training_arguments,
        data_collator=data_collator
)
model.config.use_cache = False

In [11]:
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )).__get__(model, type(model))
model = torch.compile(model)

trainer.train()

model.save_pretrained(OUTPUT_DIR)

Step,Training Loss,Validation Loss
